<a href="https://colab.research.google.com/github/bryanbits/proxylessnas/blob/master/ProxylessNAS_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cifar2png

!cifar2png cifar100 cifar100

In [ ]:
import os
import shutil
import glob
import random

!mkdir superclasses

super_to_sub = {
    "aquatic_mammals": ["beaver", "dolphin", "otter", "seal", "whale"],
    "fish": ["aquarium_fish", "flatfish", "ray", "shark", "trout"],
    "flowers": ["orchid", "poppy", "rose", "sunflower", "tulip"],
    "food_containers": ["bottle", "bowl", "can", "cup", "plate"],
    "fruit_and_vegetables":	["apple", "mushroom", "orange", "pear", "sweet_pepper"],
    "household_electrical_devices":	["clock", "keyboard", "lamp", "telephone", "television"],
    "household_furniture": ["bed", "chair", "couch", "table", "wardrobe"],
    "insects": ["bee", "beetle", "butterfly", "caterpillar", "cockroach"],
    "large_carnivores":	["bear", "leopard", "lion", "tiger", "wolf"],
    "large_man-made_outdoor_things": ["bridge", "castle", "house", "road", "skyscraper"],
    "large_natural_outdoor_scenes": ["cloud", "forest", "mountain", "plain", "sea"],
    "large_omnivores_and_herbivores": ["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],
    "medium-sized_mammals":	["fox", "porcupine", "possum", "raccoon", "skunk"],
    "non-insect_invertebrates":	["crab", "lobster", "snail", "spider", "worm"],
    "people": ["baby", "boy", "girl", "man", "woman"],
    "reptiles":	["crocodile", "dinosaur", "lizard", "snake", "turtle"],
    "small_mammals": ["hamster", "mouse", "rabbit", "shrew", "squirrel"],
    "trees": ["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],
    "vehicles_1": ["bicycle", "bus", "motorcycle", "pickup_truck", "train"],
    "vehicles_2": ["lawn_mower", "rocket", "streetcar", "tank", "tractor"]
    }


sub_to_super = {  }

for superclass in super_to_sub.keys():
  for subclass in super_to_sub[superclass]:
    sub_to_super[subclass] = superclass

print(sub_to_super)

for subclass in sub_to_super.keys():
  dest = os.path.join("superclasses", sub_to_super[subclass])
  test_dest = os.path.join(dest, "test/" + subclass)
  train_dest = os.path.join(dest, "train/" + subclass)

  test_copy = os.path.join("./cifar100/test", subclass)
  train_copy = os.path.join("./cifar100/train", subclass)

  print(test_copy + " into " + test_dest)


  print(train_copy + " into " + train_dest)

  if not os.path.exists(test_dest):
    shutil.copytree(test_copy, test_dest)
  if not os.path.exists(train_dest):
    shutil.copytree(train_copy, train_dest)

In [ ]:
# randomly groups 2 superclasses together to search for neural networks on

superclasses = [
    "aquatic_mammals",
    "fish",
    "flowers",
    "food_containers",
    "fruit_and_vegetables",
    "household_electrical_devices",
    "household_furniture",
    "insects",
    "large_carnivores",
    "large_man-made_outdoor_things",
    "large_natural_outdoor_scenes",
    "large_omnivores_and_herbivores",
    "medium-sized_mammals",
    "non-insect_invertebrates",
    "people",
    "reptiles",
    "small_mammals",
    "trees",
    "vehicles_1",
    "vehicles_2"
]

import random 
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

all_partitions = partition(superclasses, 10)


if not os.path.exists("groups"):
  !mkdir groups

import glob

for i, partition_list in enumerate(all_partitions):
  directory_name = "group_" + str(i)
  directory_path = "groups/" + directory_name
  
  test_dest = directory_path
  train_dest = directory_path

  if not os.path.exists(directory_path):
    !mkdir $directory_path
  if not os.path.exists(test_dest):
    !mkdir $test_dest
  if not os.path.exists(train_dest):
    !mkdir $train_dest

  for superclass in partition_list:
    test_path = "superclasses/" + superclass + "/test"
    train_path = "superclasses/" + superclass + "/train"

    print(test_path + " into " + test_dest)
    print(train_path + " into " + train_dest)


      # shutil.copytree(test_path, test_dest)
    !cp -r $test_path $test_dest
    !cp -r $train_path $train_dest

    # if not os.path.exists(train_dest):
      # shutil.copytree(test_path, train_dest)
    


# for subclass in sub_to_super.keys():
#   dest = os.path.join("superclasses", sub_to_super[subclass])
#   test_dest = os.path.join(dest, "test/" + subclass)
#   train_dest = os.path.join(dest, "train/" + subclass)

#   test_copy = os.path.join("./cifar100/test", subclass)
#   train_copy = os.path.join("./cifar100/train", subclass)

#   if not os.path.exists(test_dest):
#     shutil.copytree(test_copy, test_dest)
#   if not os.path.exists(train_dest):
#     shutil.copytree(train_copy, train_dest)

In [ ]:
!git clone https://github.com/bryanbits/proxylessnas.git

In [ ]:
# full dataset

!python proxylessnas/search/cifar10_arch_search.py --path "cifar100" --valid_size 100 --target_hardware gpu8 --train_batch_size 64 --test_batch_size 64

In [ ]:
# partition dataset 1

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_0" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 2

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_1" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 3

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_2" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 4

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_3" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 5

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_4" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 6

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_5" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 7

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_6" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 8

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_7" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 9

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_8" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0

In [ ]:
# partition dataset 10

!python proxylessnas/search/cifar10_arch_search.py --path "groups/group_9" --valid_size 1000 --target_hardware flops --train_batch_size 8 --test_batch_size 20 --print_frequency 100 --warmup_epochs 0